---
title:  "Handing Missing Value usin KNN"
excerpt: "using sckit-learn or handmade Function Test"

categories:
  - Useful_Fuction
tags:
  - DL
  - __nditer__  
last_modified_at: 2020-05-22T08:06:00-05:00
---

Missing Value 를 처리하는 방식은 늘 DS 직군의 사람들에게는 숙제와 같다.  
이를 간단히도 처리할수 있지만, 좀더 나은 방법으로 처리하기위해 여러 도전을 했고, 오늘은 그 중 KNN 방법에 대해서, 알아본다.

## Reference  
- [Medium article 01](https://towardsdatascience.com/the-use-of-knn-for-missing-values-cf33d935c637)  
- [Medium article 02](https://medium.com/@amrwrites/knn-based-missing-value-imputation-using-scikit-learn-802fceb5b2ea)

In [10]:
import numpy as np
import pandas as pd
from collections import defaultdict
from scipy.stats import hmean
from scipy.spatial.distance import cdist
from scipy import stats
import numbers
# train = pd.read_csv('../dataset/titanic_train.csv')
# test = pd.read_csv('../dataset/titanic_test.csv')
train = pd.read_csv('../ML_Area/data_source/titanic_train.csv')
test = pd.read_csv('../ML_Area/data_source/titanic_test.csv')
# ML_Area\data_source

In [11]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set() # setting seaborn default for plots

In [15]:
pd.concat([train.isnull().sum(),train.dtypes],axis=1,names=[['null','dtype']])

,0,1
PassengerId,0,int64
Survived,0,int64
Pclass,0,int64
Name,0,object
Sex,0,object
Age,177,float64
SibSp,0,int64
Parch,0,int64
Ticket,0,object
Fare,0,float64


'Age'  : numeric 변수  
'Cabin','Embarked : categorical 변수

In [17]:
# !python -m pip install scikit-learn==0.23.1
# !pip show version scikit-learn

## 1. Scikit-Learn KNNImputer

In [18]:
## scikit-learn : 0.23.1 이상부터 가능
from sklearn.impute import KNNImputer

In [20]:
## 'Age','Cabin'
train.iloc[10:20]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S
11,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S
12,13,0,3,"Saundercock, Mr. William Henry",male,20.0,0,0,A/5. 2151,8.0500,NaN,S
13,14,0,3,"Andersson, Mr. Anders Johan",male,39.0,1,5,347082,31.2750,NaN,S
14,15,0,3,"Vestrom, Miss. Hulda Amanda Adolfina",female,14.0,0,0,350406,7.8542,NaN,S
15,16,1,2,"Hewlett, Mrs. (Mary D Kingcome)",female,55.0,0,0,248706,16.0000,NaN,S
16,17,0,3,"Rice, Master. Eugene",male,2.0,4,1,382652,29.1250,NaN,Q
17,18,1,2,"Williams, Mr. Charles Eugene",male,NaN,0,0,244373,13.0000,NaN,S
18,19,0,3,"Vander Planke, Mrs. Julius (Emelia Maria Vande...",female,31.0,1,0,345763,18.0000,NaN,S
19,20,1,3,"Masselmani, Mrs. Fatima",female,NaN,0,0,2649,7.2250,NaN,C


`KNNImputer` 는 오로지 뉴메릭변수에서만, 활용가능하다.

In [34]:
not_cat_df = train[[col for col in train.columns if col not in ['Cabin','Embarked']]].copy() ## Cabin 제외한 df

In [54]:
int_col = train.columns[train.dtypes != 'object'].to_list()
cat_col = train.columns[train.dtypes == 'object'].to_list()

In [56]:
imputer = KNNImputer(n_neighbors=5)
# df_filled = imputer.fit_transform(not_cat_df) ## parameter df 에 카테고리변수가 있으면 무조건 에러난다. 즉. 카테고리칼 변수는 KNN 으로 missing value 처리할 수 없다.
df_filled = imputer.fit_transform(train[int_col])

In [58]:
df_filled[10:20,:]

array([[11.    ,  1.    ,  3.    ,  4.    ,  1.    ,  1.    , 16.7   ],
       [12.    ,  1.    ,  1.    , 58.    ,  0.    ,  0.    , 26.55  ],
       [13.    ,  0.    ,  3.    , 20.    ,  0.    ,  0.    ,  8.05  ],
       [14.    ,  0.    ,  3.    , 39.    ,  1.    ,  5.    , 31.275 ],
       [15.    ,  0.    ,  3.    , 14.    ,  0.    ,  0.    ,  7.8542],
       [16.    ,  1.    ,  2.    , 55.    ,  0.    ,  0.    , 16.    ],
       [17.    ,  0.    ,  3.    ,  2.    ,  4.    ,  1.    , 29.125 ],
       [18.    ,  1.    ,  2.    , 29.8   ,  0.    ,  0.    , 13.    ],
       [19.    ,  0.    ,  3.    , 31.    ,  1.    ,  0.    , 18.    ],
       [20.    ,  1.    ,  3.    , 27.6   ,  0.    ,  0.    ,  7.225 ]])

비교해보면, 승객 18,20번이 na 였는데  
`18 : NaN -> 29.8`  
`20 : NaN -> 27.6` 으로 바뀐것을 알 수 있다.

## 2. fancyimpute KNNImputer

In [60]:
from fancyimpute import KNN

Using TensorFlow backend.


In [61]:
X_filled_knn = KNN(k=5).complete(train)

AttributeError: 'KNN' object has no attribute 'complete'

In [62]:
from missingpy import KNNImputer

ModuleNotFoundError: No module named 'missingpy'